In [1]:
! pip install https://github.com/deepmind/mathematics_dataset

     - 330.0 kB 10.5 MB/s 0:00:00
  ERROR: Cannot unpack file /tmp/pip-unpack-d0k33m9m/mathematics_dataset (downloaded from /tmp/pip-req-build-452gy25b, content-type: text/html; charset=utf-8); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-452gy25b


In [2]:
!pip install /kaggle/input/mathematics-dataset/mathematics_dataset-1.0.0.tar.gz

Processing /kaggle/input/mathematics-dataset/mathematics_dataset-1.0.0.tar.gz
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/mathematics-dataset/mathematics_dataset-1.0.0.tar.gz'



In [3]:
!pip download bitsandbytes-cuda110

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.7 MB/s eta 0:00:00
Saved ./bitsandbytes_cuda110-0.26.0.post2-py3-none-any.whl
Successfully downloaded bitsandbytes-cuda110


In [4]:
# !unzip /kaggle/input/bitbyte/bitsandbytes-0.42.0-py3-none-any.whl -d /kaggle/working/
!pip install /kaggle/input/bitbyte/bitsandbytes-0.42.0-py3-none-any.whl

Processing /kaggle/input/bitbyte/bitsandbytes-0.42.0-py3-none-any.whl


# Accuracy Try 

In [5]:
!pip install mathematics_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 3.8 MB/s eta 0:00:00


In [6]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.2 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.42.0
    Uninstalling bitsandbytes-0.42.0:
      Successfully uninstalled bitsandbytes-0.42.0


In [7]:
!pip install -U accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


In [8]:
import bitsandbytes
import accelerate
print(bitsandbytes.__version__)
print(accelerate.__version__)

0.44.0
0.34.2


In [9]:
!pip install --upgrade bitsandbytes accelerate

In [10]:
from datasets import load_dataset

ds = load_dataset("lighteval/MATH", "all")

Generating train split:   0%|          | 0/7500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import bitsandbytes
import accelerate

print(bitsandbytes.__version__)
print(accelerate.__version__)

# Configure 4-bit quantization
quant_config = BitsAndBytesConfig(load_in_4bit=True, quant_type="nf4")

# Load tokenizer and model with 4-bit quantization
tokenizer = AutoTokenizer.from_pretrained("Pra-tham/quant_deepseekmath")
model = AutoModelForCausalLM.from_pretrained("Pra-tham/quant_deepseekmath", quantization_config=quant_config)

# Load the mathematics dataset
dataset = load_dataset("lighteval/MATH", "all", split='test')

# Initialize lists to store questions, true answers, and predicted answers
questions = [item['problem'] for item in dataset]
true_answers = [item['solution'] for item in dataset]
predicted_answers = []
i = 0
# Run inference with tqdm to show progress
for question in tqdm(questions[:10], desc="Processing questions"):  # Limiting to 100 questions for quick testing
    inputs = tokenizer.encode(question, return_tensors='pt')
    outputs = model.generate(inputs, max_length=512)
    predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predicted_answers.append(predicted_answer)
    print("Question   ",question)
    print("Answer   ", predicted_answers)
    
    print("True Answer ", true_answers[i])
    i += 1

# Calculate metrics
#embedding cosine rule similarity
accuracy = accuracy_score(true_answers[:10], predicted_answers)
precision = precision_score(true_answers[:10], predicted_answers, average='weighted')
recall = recall_score(true_answers[:10], predicted_answers, average='weighted')
f1 = f1_score(true_answers[:10], predicted_answers, average='weighted')

# Print results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Print a sample of results
for question, answer, pred in zip(questions[:5], true_answers[:5], predicted_answers[:5]):
    print(f"Question: {question}")
    print(f"True Answer: {answer}")
    print(f"Predicted Answer: {pred}\n")


Unused kwargs: ['quant_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


0.44.0
0.34.2


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/opt/conda/lib/python3.10/site-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


RuntimeError: No GPU found. A GPU is needed for quantization.

# Cosine Similarity

In [ ]:
!pip install sentence-transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import bitsandbytes
import accelerate

print(bitsandbytes.__version__)
print(accelerate.__version__)

# Configure 4-bit quantization
quant_config = BitsAndBytesConfig(load_in_4bit=True, quant_type="nf4")

# Load tokenizer and model with 4-bit quantization
tokenizer = AutoTokenizer.from_pretrained("Pra-tham/quant_deepseekmath")
model = AutoModelForCausalLM.from_pretrained("Pra-tham/quant_deepseekmath", quantization_config=quant_config)

# Load the mathematics dataset
dataset = load_dataset("lighteval/MATH", "all", split='test')

# Load Sentence-BERT model for embeddings
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Initialize lists to store questions, true answers, and predicted answers
questions = [item['problem'] for item in dataset]
true_answers = [item['solution'] for item in dataset]
predicted_answers = []

# Run inference with tqdm to show progress
for question in tqdm(questions[:5], desc="Processing questions"):  # Limiting to 100 questions for quick testing
    inputs = tokenizer.encode(question, return_tensors='pt')
    outputs = model.generate(inputs, max_length=512)
    predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predicted_answers.append(predicted_answer)

# Generate embeddings for true and predicted answers
true_embeddings = embedding_model.encode(true_answers[:5], convert_to_tensor=True)
predicted_embeddings = embedding_model.encode(predicted_answers, convert_to_tensor=True)

# Calculate cosine similarities
cosine_similarities = [util.pytorch_cos_sim(true, pred).item() for true, pred in zip(true_embeddings, predicted_embeddings)]

# Print average cosine similarity
average_cosine_similarity = sum(cosine_similarities) / len(cosine_similarities)
print(f"Average Cosine Similarity: {average_cosine_similarity}")

# Print a sample of results
for question, answer, pred, cos_sim in zip(questions[:5], true_answers[:5], predicted_answers[:5], cosine_similarities[:5]):
    print(f"Question: {question}")
    print(f"True Answer: {answer}")
    print(f"Predicted Answer: {pred}")
    print(f"Cosine Similarity: {cos_sim}\n")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import bitsandbytes
import accelerate

print(bitsandbytes.__version__)
print(accelerate.__version__)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Pra-tham/quant_deepseekmath")
model = AutoModelForCausalLM.from_pretrained("Pra-tham/quant_deepseekmath", [_load_in_4bit])

# Load the mathematics dataset
dataset = load_dataset('mathematics_dataset', split='test')

# Initialize lists to store questions, true answers, and predicted answers
questions = [item['question'] for item in dataset]
true_answers = [item['answer'] for item in dataset]
predicted_answers = []

# Run inference
for question in questions[:100]:  # Limiting to 100 questions for quick testing
    inputs = tokenizer.encode(question, return_tensors='pt')
    outputs = model.generate(inputs, max_length=50)
    predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predicted_answers.append(predicted_answer)

# Calculate metrics
accuracy = accuracy_score(true_answers[:100], predicted_answers)
precision = precision_score(true_answers[:100], predicted_answers, average='weighted')
recall = recall_score(true_answers[:100], predicted_answers, average='weighted')
f1 = f1_score(true_answers[:100], predicted_answers, average='weighted')

# Print results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Print a sample of results
for question, answer, pred in zip(questions[:5], true_answers[:5], predicted_answers[:5]):
    print(f"Question: {question}")
    print(f"True Answer: {answer}")
    print(f"Predicted Answer: {pred}\n")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load model directly

tokenizer = AutoTokenizer.from_pretrained("Pra-tham/quant_deepseekmath")
model = AutoModelForCausalLM.from_pretrained("Pra-tham/quant_deepseekmath")

# Load a relevant dataset (replace 'math_dataset' with your actual dataset)
dataset = load_dataset('mathematics_dataset', split='test')

# Run inference
results = []
true_answers = []
predicted_answers = []

for sample in dataset:
    question = sample['Question']
    true_answer = sample['Answer']
    inputs = tokenizer.encode(question, return_tensors='pt')
    outputs = model.generate(inputs, max_length=50)
    predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    results.append((question, predicted_answer))
    true_answers.append(true_answer)
    predicted_answers.append(predicted_answer)

# Calculate metrics
accuracy = accuracy_score(true_answers, predicted_answers)
precision = precision_score(true_answers, predicted_answers, average='weighted')
recall = recall_score(true_answers, predicted_answers, average='weighted')
f1 = f1_score(true_answers, predicted_answers, average='weighted')

# Print results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Print a sample of results
for question, answer in results[:5]:
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")
